In [37]:
pdf_file_name = "Alaska Airlines and Horizon Air SMS Manual.pdf"
#print(pdf_file_name.split(".")[0].replace(" ","_"))
index_name = str(pdf_file_name.split(".")[0].lower().replace(" ","-") + "-index")
print(index_name)

alaska-airlines-and-horizon-air-sms-manual-index


In [12]:
#!pip install azure-search-documents==11.4.0b8
#!pip install azure-identity
#!pip install openai
#!pip install langchain

In [38]:
import openai
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch
from azure.core.credentials import AzureKeyCredential
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import AzureBlobStorageContainerLoader
#from dotenv import load_dotenv

from langchain.vectorstores.azuresearch import AzureSearch
from langchain.document_loaders import AzureBlobStorageContainerLoader
from langchain.text_splitter import CharacterTextSplitter
from azure.storage.blob import BlobServiceClient
from langchain.embeddings.openai import OpenAIEmbeddings
#from dotenv import dotenv_values
import openai
import os
#from azure.search.documents.indexes import *

openai.api_key = "ea0464cc-8e3d-4eda-a0d4-9be234678a2f" 

In [39]:
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_BASE"] = "https://testingchat.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "e8143eabb02541259054426630db12a7"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
model: str = "text-embedding-ada-002"
pdf_file_name = "Alaska Airlines and Horizon Air SMS Manual.pdf"   
#e8143eabb02541259054426630db12a7    

#vector_store_address: str = "https://genaisafety.search.windows.net"
#vector_store_password: str = "iDXpfwNsnwyN88dYUlj3c0TaXP76z1DlWlSxZ9QmPxAzSeAu7ppU"

AZURE_SEARCH_SERVICE_ENDPOINT = "https://genaisafety.search.windows.net"
AZURE_SEARCH_ADMIN_KEY = "jA50VnyT0OAeuHpY0RxJGz7z9kfrmdj7YF5PHYiPXjAzSeBS6hJg"

#Creating Serach index

AZURE_SEARCH_INDEX_NAME = str(pdf_file_name.split(".")[0].lower().replace(" ","-") + "-index")

In [40]:
#pip install --upgrade azure-search-documents
embeddings = OpenAIEmbeddings(deployment = model,chunk_size=1) 
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=AZURE_SEARCH_SERVICE_ENDPOINT,
    azure_search_key=AZURE_SEARCH_ADMIN_KEY,
    index_name = AZURE_SEARCH_INDEX_NAME,
    embedding_function=embeddings.embed_query,
)
    
    

In [41]:
#%pip install azure-search-documents --pre
#%pip show azure-search-documents

import os
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient 
from azure.search.documents import SearchClient
from azure.search.documents.indexes.models import (
    ComplexField,
    VectorSearch,
    HnswVectorSearchAlgorithmConfiguration,
    SearchField,
    CorsOptions,
    SearchIndex,
    ScoringProfile,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
)
from azure.core.exceptions import ResourceNotFoundError
import tempfile
from langchain.document_loaders import PyPDFLoader
#pdf_file_name = "Alaska Airlines and Horizon Air SMS Manual.pdf"

from azure.storage.blob import BlobServiceClient
from azure.search.documents import IndexDocumentsBatch

In [42]:
def pdf_text_extraction(pdf_file_path):
    if pdf_file_path is not None:
        # Open the PDF file and read its contents
        with open(pdf_file_path, 'rb') as pdf_file:
            # Create a temporary file to write the PDF contents
            with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
                tmp_file.write(pdf_file.read())
                pdf_path = tmp_file.name

        # Now you can use the PyPDFLoader on the temporary PDF file
        loader = PyPDFLoader(pdf_path)
        pages_split = loader.load_and_split()
    return pages_split

In [46]:
# Replace with your actual connection string
connection_string = "DefaultEndpointsProtocol=https;AccountName=storageaccountgenai;AccountKey=WAfyL4uqWgarFdx1ibNgWv9lmOINODLuN6nnLSQLgE/iuHhKGi1pYd6NQJ6LBnZO/DnnQfhbNSWi+AStsOLf1Q==;EndpointSuffix=core.windows.net"

# Replace with your container name
container_name = "safety"

# Create a BlobServiceClient using the connection string
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Get a reference to the container
container_client = blob_service_client.get_container_client(container_name)

# List blobs in the container
blobs = container_client.list_blobs()
dict_data = []
for blob in blobs:
    #print(blob)
    if blob.name.startswith("manuals") and blob.name.endswith(".pdf"):  # Check if the blob is a PDF file
        blob_client = container_client.get_blob_client(blob)
        print(blob.name)
        try:
            pdf_files = blob_client.download_blob().readall()
                # Save the blob content to a local PDF file or process it as needed
            pdf_name = blob.name.split('/')[1]
            print(pdf_name)
            with open(pdf_name, "wb") as pdf_file:
                pdf_file.write(pdf_files)
                print("PDF file downloaded successfully.")
        except Exception as e:
             print(f"An error occurred: {e}")       
            
        
        #pdf_file_name=str(blob.name).split(".")[0]
        #pdf_name = "AAG_Safety_Manual.pdf"
        pdf_name = pdf_file_name

        #loader = PyPDFLoader(pdf_name)
        loader = pdf_text_extraction(pdf_name)
        
        documents = loader #.load()
        #print(documents)
        text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
        docs = text_splitter.split_documents(documents)
        print(f"splitted into {len(docs)} documents") 
        
        #vector_store.add_documents(documents=docs)
        #print(docs)
        

        credential = AzureKeyCredential(AZURE_SEARCH_ADMIN_KEY)
        search_client = SearchIndexClient(endpoint=AZURE_SEARCH_SERVICE_ENDPOINT, credential=credential)
        search_client_v2 = SearchClient(endpoint=AZURE_SEARCH_SERVICE_ENDPOINT, index_name=AZURE_SEARCH_INDEX_NAME, credential=credential)

        bid=str(blob.name).split("/")[1].split(".")[0].lower().replace(" ","_").replace("_","-")
        AZURE_SEARCH_INDEX_NAME=bid + "-index"
        print(AZURE_SEARCH_INDEX_NAME)
        # Define your Azure Cognitive Search credentials and index settings as previously done
        # Check if the index already exists
        existing_index = None
        # Define your Azure Cognitive Search credentials and index settings as previously done
        
        
        for doc in docs:
            dict_data.append({
                'id': blob.name,
                'page_content': doc.page_content,  # You can set a title based on your needs
                'page_number': doc.metadata['page'],
            })

        print(f"splitted into {len(docs)} documents") 
        
        # Generate embeddings for title and content fields
        for item in dict_data:
            title = item['id']
            content = item['page_content']
            title_embeddings = embeddings.embed_query(title)
            content_embeddings = embeddings.embed_query(content)
            item['titleVector'] = title_embeddings
            item['contentVector'] = content_embeddings
            
        print('111',dict_data)

        #final_doc=[]
        #for index,doc in enumerate(docs):
        #   final_doc.append({'id':bid,'content':doc.page_content,'metadata':doc.metadata['source'], 'contentVector':content_embeddings[index]})
        #print(final_doc,'!!')
        # Generate embeddings for title and content fields   
            
            
            # The index doesn't exist, so create a new one
        if existing_index == None:
            #index = SearchIndex(name=AZURE_SEARCH_INDEX_NAME,fields=[SimpleField(name="id", type=SearchFieldDataType.String, key=True, searchable=False),SearchableField(name="content", type=SearchFieldDataType.String, searchable=True),SearchableField(name="content_vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single), searchable=True, vector_search_dimensions=1536,vector_search_configuration="my-vector-config",),SearchableField(name="metadata", type=SearchFieldDataType.String, searchable=True),],)
            #index = SearchIndex(name=AZURE_SEARCH_INDEX_NAME,fields=[SimpleField(name="id", type=SearchFieldDataType.String, key=True, searchable=False),SearchableField(name="content", type=SearchFieldDataType.String, searchable=True),SearchableField(name="metadata", type=SearchFieldDataType.String, searchable=True),],)
            fields = [ 
                SimpleField(name = "id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True),
                SearchableField(name = 'content', type=SearchFieldDataType.String),
                SearchableField(name='metadata', type=SearchFieldDataType.String),
                SearchField(name="contentVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                        searchable=True, vector_search_dimensions=1536, vector_search_configuration="my-vector-config"),

            ]
            index = SearchIndex(name=AZURE_SEARCH_INDEX_NAME, fields = fields,)

            # Create the new index
            search_client.create_or_update_index(index)
            #print(f"Index '{AZURE_SEARCH_INDEX_NAME}' created successfully.")
            result = search_client_v2.upload_documents(documents=dict_data)
            print("Vector store added successfully")

        else:
            try:
                # Check if the index exists
                existing_index = search_client.get_index(AZURE_SEARCH_INDEX_NAME)
                print(f"Index '{AZURE_SEARCH_INDEX_NAME}' already exists. You can use the existing index.")
                #vector_store.add_documents(documents=final_doc)
                result = search_client_v2.upload_documents(documents=dict_data)
                print(f"Uploaded {len(content)} documents")
                print("Vector store added successfully")
            except Exception as e:
                print(e)
                pass
        #document = [{"id": bid, "content": docs}]
        #document = [{"id": doc["id"], "content": doc["content"]}]
        #document = [{"id": doc.get_id(), "content": doc.get_content()}]


        # Submit the documents for indexing
        #result = search_client.upload_documents(documents=final_do)
        #vector_store.add_documents(documents=final_doc)
        #print("Vector store added successfully")

manuals/AAG_Safety_Manual.pdf
AAG_Safety_Manual.pdf
PDF file downloaded successfully.
splitted into 125 documents
aag-safety-manual-index
splitted into 125 documents


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



HttpResponseError: (InvalidRequestParameter) The request is invalid. Details: definition : The field 'contentVector' uses a vector search algorithm configuration 'my-vector-config' which is not defined.
Code: InvalidRequestParameter
Message: The request is invalid. Details: definition : The field 'contentVector' uses a vector search algorithm configuration 'my-vector-config' which is not defined.
Exception Details:	(UnknownVectorAlgorithmConfiguration) The field 'contentVector' uses a vector search algorithm configuration 'my-vector-config' which is not defined. Parameters: definition
	Code: UnknownVectorAlgorithmConfiguration
	Message: The field 'contentVector' uses a vector search algorithm configuration 'my-vector-config' which is not defined. Parameters: definition

In [19]:
blob.name = "manuals/Alaska Airlines and Horizon Air SMS Manual.pdf" #delete this later and merge the below and above cells

In [20]:
#embeddings_result = embeddings.embed_query(text)

for item in docs:
    content = item.page_content
    content_embeddings = embeddings.embed_query(content)
    #item['contentVector'] = content_embeddings

In [29]:

vector_search = VectorSearch(
    algorithm_configurations=[
        HnswVectorSearchAlgorithmConfiguration(
            name="my-vector-config",
            kind="hnsw",
            parameters={
                "m": 4,
                "efConstruction": 400,
                "efSearch": 500,
                "metric": "cosine"
            }
        )
    ]
)

In [32]:
docs[0]

Document(page_content='Uncontrolled copy when downloaded or printed from an electronic document.\nSafety Management System \n(SMS) Manual\nAlaska Airlines, Inc. / Horizon Air\n(Alaska Air Group)\nRevision 18 6/16/2023', metadata={'source': '/var/folders/g1/zd40wpl1023dlyqyjv2xplcc0000gq/T/tmpjth72l2u', 'page': 0})

In [ ]:
# Pure Vector Search
from azure.search.documents.models import Vector
query = "software development"  
  
vector = Vector(value=generate_embeddings(query), k=3, fields="contentVector")
  
results = search_client_v2.search(  
    search_text=None,  
    vectors= [vector],
    select=["id", "content", "metadata"],
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  

In [ ]:
# Perform a similarity search
f = vector_store.similarity_search(
    query="what is requirement for responsible ai?",
    k=3,
    search_type="similarity",
)
print(f[0].page_content)

In [ ]:
blob.name

In [ ]:
# TODO: Handle the case where the user doesn't provide ids on the Collection
texts = [doc.page_content for doc in docs]
#print(texts)
metadatas = [doc.metadata for doc in docs]
print(metadatas[0])

In [ ]:
len(docs)
res=[]
for doc in docs:
    res.append({'id':bid,'content':doc.page_content,'metadata':doc.metadata})

In [ ]:
print(res)

In [ ]:
type(docs)

In [ ]:
pip install --upgrade azure-search-documents

In [ ]:
docs[0]

In [ ]:
def extract_text_from_pdf(pdf_stream):
    text = ""
    try:
        pdf_reader = PyPDF2.PdfFileReader(pdf_stream)
        num_pages = pdf_reader.numPages
        for page_num in range(num_pages):
            page = pdf_reader.getPage(page_num)
            text += page.extractText()
    except Exception as e:
        print(f"Error extracting text from PDF: {str(e)}")
    return text

pdf_text = extract_text_from_pdf(pdf_stream)


In [ ]:
index = SearchIndex(
    name=AZURE_SEARCH_INDEX_NAME,
    fields=[
        SimpleField(name="id", type=SearchFieldDataType.String, key=True, searchable=False),
        SearchableField(name="content", type=SearchFieldDataType.String, searchable=True),
    ],
)
index
#search_client.create_index(index)


try:
    result = search_client.get_index(index_name)
    print(f"Index '{index_name}' created successfully.")
except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
import PyPDF2

def extract_text_from_pdf(pdf_file_path):
    text = ""
    try:
        with open(pdf_file_path, 'rb') as pdf_file:
            pdf_reader = PyPDF2.PdfFileReader(pdf_file)
            num_pages = pdf_reader.numPages
            for page_num in range(num_pages):
                page = pdf_reader.getPage(page_num)
                text += page.extractText()
    except Exception as e:
        print(f"Error extracting text from PDF: {str(e)}")
    return text


# Initialize the SearchIndexClient and SearchClient
credential = AzureKeyCredential(AZURE_SEARCH_ADMIN_KEY)
search_client = SearchIndexClient(endpoint=AZURE_SEARCH_SERVICE_ENDPOINT, credential=credential)
search_client_v2 = SearchClient(endpoint=AZURE_SEARCH_SERVICE_ENDPOINT, index_name=AZURE_SEARCH_INDEX_NAME, credential=credential)
# Replace with your actual connection string
connection_string = "DefaultEndpointsProtocol=https;AccountName=storageaccountgenai;AccountKey=WAfyL4uqWgarFdx1ibNgWv9lmOINODLuN6nnLSQLgE/iuHhKGi1pYd6NQJ6LBnZO/DnnQfhbNSWi+AStsOLf1Q==;EndpointSuffix=core.windows.net"
# Create a BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
# Replace with your container name
container_name = "safety"
# Get a reference to the container
container_client = blob_service_client.get_container_client(container_name)

# List blobs in the container
blobs = container_client.list_blobs()
for blob in blobs:
    #print(blob)
    if blob.name.startswith("manuals") and blob.name.endswith(".pdf"):  # Check if the blob is a PDF file
        blob_client = container_client.get_blob_client(blob)
        pdf_files = blob_client.download_blob().readall()
        print(type(pdf_files))
        pdf_file_name=str(blob.name).split(".")[0]
        loader = AzureBlobStorageContainerLoader(conn_str=connection_string,container=container_name,prefix=pdf_file_name,)
        documents = loader.load()
        text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
        docs = text_splitter.split_documents(documents)
        print(docs)
        # Define the index with 'id' and 'content' fields
        bid=str(blob.name).split("/")[1].split(".")[0].lower().replace(" ","_").replace("_","-")
        AZURE_SEARCH_INDEX_NAME=bid + "-index"
        index = SearchIndex(name=AZURE_SEARCH_INDEX_NAME,fields=[SimpleField(name="id", type="Edm.String", key=True, searchable=False),SearchableField(name="content", type="Edm.String", searchable=True),],)
        # Create the index if it doesn't exist
        existing_index = None
        # Define your Azure Cognitive Search credentials and index settings as previously done
        try:
            # Check if the index exists
            existing_index = search_client.get_index(AZURE_SEARCH_INDEX_NAME)
            print(f"Index '{AZURE_SEARCH_INDEX_NAME}' already exists. You can use the existing index.")
        except Exception as e:
            pass
            # The index doesn't exist, so create a new one
        if existing_index == None:
            index = SearchIndex(name=AZURE_SEARCH_INDEX_NAME,fields=[SimpleField(name="id", type=SearchFieldDataType.String, key=True, searchable=False),SearchableField(name="content", type=SearchFieldDataType.String, searchable=True),],)
            # Create the new index
            search_client.create_index(index)
            print(f"Index '{AZURE_SEARCH_INDEX_NAME}' created successfully.")
        #document = [{"id": bid, "content": docs}]
        #document = [{"id": doc["id"], "content": doc["content"]}]
        #document = [{"id": doc.get_id(), "content": doc.get_content()}]
        documents = [{"id": bid, "content": pdf_text}]
        search_client_v2.upload_documents(documents=documents)


In [ ]:
blobs = container_client.list_blobs()
for blob in blobs:
    if blob.name.startswith("manuals") and blob.name.endswith(".pdf"):  # Check if the blob is a PDF file
        print('yes')
        blob_client = container_client.get_blob_client(blob)
        pdf_files = blob_client.download_blob().readall()
        pdf_file_name=str(blob.name).split("/")[1]
        print(pdf_file_name)
        pdf_text = extract_text_from_pdf(pdf_file_name)
        print(pdf_text)

In [ ]:
##########################CODE BASE#########################
credential = AzureKeyCredential(AZURE_SEARCH_ADMIN_KEY)
search_client = SearchIndexClient(endpoint=AZURE_SEARCH_SERVICE_ENDPOINT, credential=credential)

#embeddings: OpenAIEmbeddings = OpenAIEmbeddings(deployment="textembeddingada", model = "text-embedding-ada-002", chunk_size=1)
embeddings = OpenAIEmbeddings(deployment = model,chunk_size=1) 
#vector_store: AzureSearch = AzureSearch(
#    azure_search_endpoint=AZURE_SEARCH_SERVICE_ENDPOINT,
#    azure_search_key=AZURE_SEARCH_ADMIN_KEY,
#    index_name = AZURE_SEARCH_INDEX_NAME,
#    embedding_function=embeddings.embed_query,
#)

def pdf_text_extraction(pdf_file_path):
    if pdf_file_path is not None:
        # Open the PDF file and read its contents
        with open(pdf_file_path, 'rb') as pdf_file:
            # Create a temporary file to write the PDF contents
            with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
                tmp_file.write(pdf_file.read())
                pdf_path = tmp_file.name

        # Now you can use the PyPDFLoader on the temporary PDF file
        loader = PyPDFLoader(pdf_path)
        pages_split = loader.load_and_split()
    return pages_split

import tempfile
from langchain.document_loaders import PyPDFLoader
#pdf_file_name = "Alaska Airlines and Horizon Air SMS Manual.pdf"

from azure.storage.blob import BlobServiceClient

# Replace with your actual connection string
connection_string = "DefaultEndpointsProtocol=https;AccountName=storageaccountgenai;AccountKey=WAfyL4uqWgarFdx1ibNgWv9lmOINODLuN6nnLSQLgE/iuHhKGi1pYd6NQJ6LBnZO/DnnQfhbNSWi+AStsOLf1Q==;EndpointSuffix=core.windows.net"
# Create a BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Replace with your container name
container_name = "safety"

# Get a reference to the container
container_client = blob_service_client.get_container_client(container_name)

# List blobs in the container
blobs = container_client.list_blobs()

for blob in blobs:
    #print(blob)
    if blob.name.startswith("manuals") and blob.name.endswith(".pdf"):  # Check if the blob is a PDF file
        blob_client = container_client.get_blob_client(blob)
        pdf_files = blob_client.download_blob().readall()  
        #pages = pdf_text_extraction(pdf_file_name)
        #print(blob)
        loader = PyPDFLoader(pdf_file_name)
        documents = loader.load()
        text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
        docs = text_splitter.split_documents(documents)
        bid=str(blob.name).split("/")[1].split(".")[0].lower().replace(" ","_").replace("_","-")
        AZURE_SEARCH_INDEX_NAME=bid + "-index"
        print(AZURE_SEARCH_INDEX_NAME)
        # Define your Azure Cognitive Search credentials and index settings as previously done
        # Check if the index already exists
        existing_index = None
        # Define your Azure Cognitive Search credentials and index settings as previously done
        try:
            # Check if the index exists
            existing_index = search_client.get_index(AZURE_SEARCH_INDEX_NAME)
            print(f"Index '{AZURE_SEARCH_INDEX_NAME}' already exists. You can use the existing index.")
        except Exception as e:
            pass
            # The index doesn't exist, so create a new one
        if existing_index == None:
            index = SearchIndex(name=AZURE_SEARCH_INDEX_NAME,fields=[SimpleField(name="id", type=SearchFieldDataType.String, key=True, searchable=False),SearchableField(name="content", type=SearchFieldDataType.String, searchable=True),],)
            # Create the new index
            search_client.create_index(index)
            print(f"Index '{AZURE_SEARCH_INDEX_NAME}' created successfully.")
        #document = [{"id": bid, "content": docs}]
        #document = [{"id": doc["id"], "content": doc["content"]}]
        document = [{"id": doc.get_id(), "content": doc.get_content()}]
        #print(documents)
        # Add the documents to the Azure Cognitive Search index
        vector_store: AzureSearch = AzureSearch(azure_search_endpoint=AZURE_SEARCH_SERVICE_ENDPOINT,azure_search_key=AZURE_SEARCH_ADMIN_KEY,index_name = AZURE_SEARCH_INDEX_NAME,embedding_function=embeddings.embed_query,)
        vector_store.add_documents(documents=document)
        #print("vector Created Successfully.")

            # Continue with the rest of your code, such as extracting text from PDFs and adding to the index
        #search_client.create_index(index)
        #docs
        #vector_store.add_documents(documents=docs)
        

#blobs = container_client.list_blobs()
#for blob in blobs:
#    print(blob.name)

#loader = PyPDFLoader("Alaska Airlines and Horizon Air SMS Manual.pdf")
#pages = loader.load_and_split()

#documents = loader.load()


In [ ]:
vector_store.add_documents(documents=docs)

# Perform a similarity search
docs = vector_store.similarity_search(
    query="safety Actions",
    k=3,
    search_type="similarity",
)
print(docs[0].page_content)
docs



In [ ]:
--end--
